In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
from torch import nn

## Access src directory from ./notebooks/ folder
from pathlib import Path
sys.path.append(str(Path('.').absolute().parent))

from src.Snip import SNIP
from src.models.LeNet import LeNet300100, LeNet5Caffe
from src.optimization import MeanEvaluator

import numpy as np
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST, CIFAR10
import torchvision.transforms as transforms

In [ ]:
prune_model = LeNet5Caffe()
snip = SNIP(prune_model)
total_param_number = snip.get_total_param_number()
print(total_param_number)

In [ ]:
del evaluator

In [ ]:
evaluator = MeanEvaluator(model_class = LeNet5Caffe,
                         dataset = MNIST,
                         eval_n = 10,
                         epochs = 20,
                         pruning_ratio = 0.02)

#baseline_score = evaluator.evaluate_baseline()
#print(baseline_score)

snip_score = evaluator.evaluate_pruned_model()
print(1 - snip_score[0])

#diff = baseline_score - snip_score
#print(diff)

In [ ]:
evaluator = MeanEvaluator(model_class = LeNet5Caffe,
                         dataset = MNIST,
                         eval_n = 10,
                         epochs = 20,
                         pruning_ratio = 0.01)

#baseline_score = evaluator.evaluate_baseline()
#print(baseline_score)

snip_score = evaluator.evaluate_pruned_model()
print(snip_score)

In [ ]:
evaluator = MeanEvaluator(model_class = LeNet300100,
                         dataset = MNIST,
                         eval_n = 10,
                         epochs = 15,
                         pruning_ratio = 0.02)

#baseline_score = evaluator.evaluate_baseline()
#print(baseline_score)

snip_score = evaluator.evaluate_pruned_model()
print(snip_score)

In [ ]:
evaluator = MeanEvaluator(model_class = LeNet300100,
                         dataset = MNIST,
                         eval_n = 10,
                         epochs = 15,
                         pruning_ratio = 0.05)

#baseline_score = evaluator.evaluate_baseline()
#print(baseline_score)

snip_score = evaluator.evaluate_pruned_model()
print(1 - snip_score[0])